In [2]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
device_u = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_v = torch.device('cuda:1' if torch.cuda.device_count() > 1 else device_u)
device_p = torch.device('cuda:2' if torch.cuda.device_count() > 2 else device_u)

print(f"device_u: {device_u}")
print(f"device_v: {device_v}")
print(f"device_p: {device_p}")

device_u: cuda:0
device_v: cuda:1
device_p: cuda:2


In [5]:
class FCN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_layers = 4, neurons = 50):
        super(FCN, self).__init__()
        layers = [nn.Linear(in_dim, neurons), nn.Tanh()]
        for _ in range(hidden_layers - 1):
            layers += [nn.Linear(neurons, neurons), nn.Tanh()]
        layers.append(nn.Linear(neurons, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self,x):
        return self.net(x)
    
net_u = FCN(in_dim = 2, out_dim = 1).to(device_u)
net_v = FCN(in_dim = 2, out_dim = 1).to(device_v)
net_p = FCN(in_dim = 2, out_dim = 1).to(device_p)

In [6]:
def generate_collocation_points(N):
    x = torch.rand(N,1)
    y = torch.rand(N,1)

    X = torch.cat([x,y], dim = 1)
    return X

N_f = 10000

X_f = generate_collocation_points(N_f)

X_f_u = X_f.clone().detach().to(device_u).requires_grad_(True)
X_f_v = X_f.clone().detach().to(device_v).requires_grad_(True)
X_f_p = X_f.clone().detach().to(device_p).requires_grad_(True)


In [7]:
def generate_boundary_points(N):
    x_l = torch.zeros(N,1) ; x_r = torch.ones(N,1); ylr = torch.rand(N,1)
    b_l = torch.cat([x_l,ylr], dim = 1); b_r = torch.cat([x_r,ylr], dim = 1)

    y_b = torch.zeros(N,1); y_t = torch.ones(N,1); xbt = torch.rand(N,1)
    b_b = torch.cat([xbt,y_b], dim = 1); b_t = torch.cat([xbt,y_t], dim = 1)

    b = torch.cat([b_l,b_r,b_b,b_t], dim = 0)

    return b

N_b = 2000

X_b = generate_boundary_points(N_b)
X_b_u = X_b.clone().detach().to(device_u)
X_b_v = X_b.clone().detach().to(device_v)

In [9]:
X_p = torch.tensor([[0.0,0.0]],dtype=torch.float32)



In [10]:
def compute_residual_u():
    # Evaluate u and its derivatives on device_u
    u = net_u(X_f_u)
    grads_u = autograd.grad(u, X_f_u, grad_outputs=torch.ones_like(u),create_graph= True )[0]

    u_x = grads_u[:,0:1]
    u_y = grads_u[:,1:2]

    u_x_x = autograd.grad(u_x, X_f_u, grad_outputs=torch.ones_like(u_x),create_graph= True )[0][:,0:1]
    u_y_y = autograd.grad(u_y, X_f_u, grad_outputs=torch.ones_like(u_y),create_graph= True )[0][:,1:2]

    # Get v and p as constants on device_u
    v = net_v(X_f_v).detach().to(device_u)
    p = net_p(X_f_p)

    # Spatial derivatives of pressure
    p_x = autograd.grad(p, X_f_p, grad_outputs=torch.ones_like(p),create_graph= True,allow_unused= True )[0][:,0:1].to(device_u)

    nu = 1.00/400

    res_u = u * u_x + v * u_y + p_x - nu * (u_x_x + u_y_y)

    return res_u


In [12]:
def compute_residual_v():
    # Evaluate v and its derivatives on device_v
    v = net_v(X_f_v)
    grads_v = autograd.grad(v, X_f_v, grad_outputs=torch.ones_like(v),create_graph= True )[0]

    v_x = grads_v[:,0:1]
    v_y = grads_v[:,1:2]

    v_x_x = autograd.grad(v_x, X_f_v, grad_outputs=torch.ones_like(v_x),create_graph= True )[0][:,0:1]
    v_y_y = autograd.grad(v_y, X_f_v, grad_outputs=torch.ones_like(v_y),create_graph= True )[0][:,1:2]

    # Get u and p as constants on device_v
    u = net_u(X_f_u).detach().to(device_v)
    p = net_p(X_f_p)

    # Spatial derivatives of pressure
    p_y = autograd.grad(p, X_f_p, grad_outputs=torch.ones_like(p),create_graph= True,allow_unused= True )[0][:,1:2].to(device_v)

    nu = 1.00/400

    res_v = u * v_x + v * v_y + p_y - nu * (v_x_x + v_y_y)

    return res_v

In [13]:
def compute_residual_continuity():
    # Divergence of velocity field on GPU 0
    u = net_u(X_f_u)
    v = net_v(X_f_v)
    u_x = autograd.grad(u, X_f_u, grad_outputs=torch.ones_like(u), create_graph=True)[0][:, 0:1]
    v_y = autograd.grad(v, X_f_v, grad_outputs=torch.ones_like(v), create_graph=True)[0][:, 1:2]
    res_c = u_x + v_y.to(device_u)
    return res_c